In [ ]:
!pip install telebot
!pip install asyncio
!pip install requests
!pip install threading
!pip install base64

In [ ]:
import telebot
import asyncio
import requests
import base64

cantidad = 1
hiresfix = False
adetailer = False
TOKEN = '6318548166:AAHpw7I4gFc2I8a8KZ4rPnLa8B_bnHXvZ4Y'
# 6537981587:AAHsIt2nWXWyjvGTKrPElYBT0rpMUpArl5c
# 6318548166:AAHpw7I4gFc2I8a8KZ4rPnLa8B_bnHXvZ4Y
bot = telebot.TeleBot(TOKEN)

estado = False

def get_keyboard(tipo):
    global estado
    if tipo == 'principal':
        keyboard_principal = telebot.types.InlineKeyboardMarkup()
        if estado:
            button_generar = telebot.types.InlineKeyboardButton(text="Generando", callback_data="imagine_callback")
        else:
            button_generar = telebot.types.InlineKeyboardButton(text="Generar", callback_data="imagine_callback")
        button_cantidad = telebot.types.InlineKeyboardButton(text=f"Cantidad [{str(cantidad)}]", callback_data="cantidad_callback")
        men_adetailer = "On ✅" if adetailer else "Off ❌"
        button_adetailer = telebot.types.InlineKeyboardButton(text=f"Adetailer [{men_adetailer}]", callback_data="adetailer_callback")
        men_hiresfix = "On ✅" if hiresfix else "Off ❌"
        button_hrfix = telebot.types.InlineKeyboardButton(text=f"HiRes Fix [{men_hiresfix}]", callback_data="hrfix_callback")
        keyboard_principal.add(button_generar, button_cantidad, button_adetailer, button_hrfix)
        return keyboard_principal


def generate_images(call, message):
    global cantidad
    global hiresfix
    global adetailer
    global estado
    payload = {
        "prompt": message.text.split('\n')[2],
        "negative_prompt": "(low quality, worst quality:1.4), cgi, text, signature, watermark, extra limbs, ((nipples))",
        "steps": 10,
        "cfg_scale": 2,
        "n_iter": cantidad,
        "width": 1024,
        "height": 1024,
        "sampler_name": 'DPM++ SDE Karras',
        "enable_hr": hiresfix,
        "adetailer": adetailer,
        "hr_scale": 2,
        "denoising_strength": 0.5,
        "hr_upscaler": '4x_NMKD-Superscale-SP_178000_G',
        "alwayson_scripts": '',
    }
    print(payload)
    if payload["enable_hr"]:
        payload["hr_second_pass_steps"] = round(payload["steps"] / 2)
    if payload["adetailer"]:
        payload["alwayson_scripts"] = {
            "ADetailer": {
                "args": [
                    {"ad_model": "mediapipe_face_full", "ad_confidence": 0.35},
                    {"ad_model": "mediapipe_face_mesh_eyes_only", "ad_confidence": 0.35},
                ]
            }
        }
    # print(payload)
    modelo = requests.get(url=f'http://localhost:7860/sdapi/v1/sd-models')  
    modelo = modelo.json()[0]["model_name"]
    
    if 'dreamshaperXL' in modelo:
        payload["width"] = 1024
        payload["height"] = 1024
        # payload["prompt"] += ". <lora:add-detail-xl:1>, <lora:xl_more_art-full_v1:1>"
        # payload["negative_prompt"] = "(low quality, worst quality:1.4), cgi, text, signature, watermark, extra limbs, ((nipples))"
        payload["steps"] = 8
        payload["cfg_scale"] = 2
        payload["sampler_name"] = 'DPM++ SDE Karras'
    
    if "r_color" in payload["prompt"]:
        payload["prompt"] = payload["prompt"].replace('r_color', " {Ruby Red|Sapphire Blue|Emerald Green|Golden Yellow|Amethyst Purple|Crimson Red|Ocean Blue|Forest Green|Sunflower Yellow|Royal Purple|Coral Pink|Turquoise Blue|Olive Green|Topaz Yellow|Lavender Purple|Cherry Blossom Pink|Mint Green|Butterscotch Yellow|Indigo Blue|Pine Green|Marigold Orange|Periwinkle Blue|Moss Green|Tangerine Orange|Teal Blue|Burgundy Red|Sky Blue|Chartreuse Green|Mustard Yellow|Mulberry Purple|Salmon Pink|Steel Gray|Peachy Orange|Cobalt Blue|Jade Green|Lemon Yellow|Mauve Purple|Apricot Orange|Cerulean Blue|Chartreuse Yellow|Plum Purple|Peachy Pink|Minty Blue-Green|Mahogany Brown|Azure Blue|Lilac Purple|Caramel Brown|Cyan Blue|Rose Gold|Midnight Black|Pearlescent White|}")
    if "r_artist" in payload["prompt"]:
        payload["prompt"] = payload["prompt"].replace('r_artist', "{Art by Hokusai|Art by Utagawa Hiroshige|Art by Katsushika Oei|Art by Ando Hiroshige|Art by Hiroshi Yoshida|Art by Katsushika Hokusai II|Art by Kanō Tanyū|Art by Kawanabe Kyōsai|Art by Gustave Doré|Art by Alphonse Mucha|Art by Hajime Sorayama|Art by Eyvind Earle|Art by H.R. Giger|Art by Ilya Kuvshinov|Art by Syd Mead|Art by Simon Stålenhag|Art by Boris Vallejo|Art by Roger Dean|Art by Yoko Honda|Art by James Jean|Art by Claude Monet|Art by Vincent van Gogh|Art by Pablo Picasso|Art by Frida Kahlo|Art by Salvador Dalí|Art by Georgia O'Keeffe|Art by Wassily Kandinsky|Art by Jackson Pollock|Art by Andy Warhol|Art by Leonardo da Vinci|Art by Michelangelo|Art by Raphael|Art by Sandro Botticelli|Art by Caravaggio|Art by Rembrandt|Art by Johannes Vermeer|Art by Pieter Bruegel the Elder|Art by Hieronymus Bosch|Art by Titian|Art by Tintoretto|Art by El Greco|Art by Peter Paul Rubens|Art by Jan van Eyck|Art by Francisco Goya|Art by Diego Velázquez|Art by Édouard Manet|Art by Pierre-Auguste Renoir|Art by Amedeo Modigliani|Art by Egon Schiele|Art by Kazimir Malevich|Art by Yayoi Kusama|Art by KAWS|Art by Banksy|Art by Jeff Koons|Art by Ai Weiwei|Art by Yves Klein|Art by Jean-Michel Basquiat|Art by Takashi Murakami|Art by Marina Abramović|Art by Anish Kapoor|Art by Damien Hirst|Art by Jenny Holzer|Art by Olafur Eliasson|Art by David Hockney|Art by Bridget Riley|Art by Antony Gormley|Art by Cindy Sherman|Art by Gerhard Richter|Art by Anselm Kiefer|Art by Barbara Hepworth|Art by Louise Bourgeois|Art by Yinka Shonibare|Art by Subodh Gupta|Art by Xu Bing|Art by Yoshitomo Nara|Art by Murakami Saburo|Art by Lee Ufan|Art by Choi Jeong Hwa|Art by Arin Dwihartanto Sunaryo|Art by Raqib Shaw|Art by Julio Le Parc|Art by Zao Wou-Ki|Art by Lee Krasner|Art by Joan Miró|Art by Max Ernst|Art by Fernand Léger|Art by Paul Klee|Art by René Magritte|Art by Giorgio de Chirico|Art by Marc Chagall}");
    if "r_style" in payload["prompt"]:
        payload["prompt"] = payload["prompt"].replace('r_style', " {Origami|Charcoal Drawing|Digital Drawing|Glass Art|Oil Painting|Acrylic Painting|Watercolor|Digital Art|Wood Sculpture|Stone Sculpture|Metal Sculpture|Collage|Etching|Screen Printing|Fine Art Photography|Graffiti|Street Art|Surrealism|Abstract Art|Cubism|Impressionism|Expressionism|Renaissance Art|Baroque Art|Romanticism|Gothic Art|Minimalism|Contemporary Art|Pop Art|Naïve Art|Realism|Conceptual Art|Kitsch Art|Art Deco|Rococo Art|Pre-Raphaelite Art|Post-Impressionism|Art Deco|Fauvism|Dadaism|Constructivism|Futurism|Rococo Art|Classical Art|Neoclassical Art|Fiber Art|Pastel Drawing|Bronze Sculpture|Ceramic Art|Ink Wash Painting|Encaustic Painting|Mosaic Art|Mixed Media Art|Plaster Sculpture|Ice Sculpture|Wire Sculpture|Assemblage|Woodcut Print|Lithography|Alternative Process Photography|Calligraphy|Stencil Art|Land Art|Digital Collage|Op Art|Surrealist Collage|Constructive Realism|Tonalism|Symbolism|Precisionism|Byzantine Art|Kinetic Art|Interactive Art|Installation Art|Environmental Art|Kinetic Sculpture|Hyperrealism|Bio Art|Steampunk Art|New Media Art|Geometric Abstraction|Superflat Art|Outsider Art|Magic Realism|Trompe-l'oeil|Metaphysical Painting|Asiatic Pictorialism|Spatialism|Transgressive Art|Folk Art|Primitive Art|Powerful Art|Vorticism|Situationist International|Classical Realism|Social Realism}")
    if "r_place" in payload["prompt"]:
        payload["prompt"] = payload["prompt"].replace('r_place', " {Moon Base Outpost|Enchanted Coral Reef|Cyberpunk Alleyway|Floating Sky Islands|Digital Virtual Reality World|Ancient Egyptian Pyramid|Jungle Canopy Treehouse|Subterranean Cave System|European Medieval Village|Underground Bunker Complex|Pacific Island Volcano|Interstellar Space Station|Arctic Iceberg Landscape|Indian Palace Courtyard|Sunken Pirate Shipwreck|Urban Graffiti Art District|Mayan Temple City|Time-Traveling Victorian Street|Roman Colosseum Arena|Surreal Salvador Dalí Landscape|Holographic Futuristic Mall|Crystal Cavern Haven|Celestial Garden Oasis|Neon Cyberpunk Floating Market|Levitating Cloud Sanctuary|Binary Code Wonderland|Stargate Portal Nexus|Bioluminescent Mushroom Forest|Subaquatic Atlantis Observatory|Steampunk Labyrinth Workshop|Aetherial Library Archive|Aurora Borealis Ice Palace|Galactic Beacon Outpost|Lost City of Echoing Whispers|Floating Lotus Palace|Deserted Time Capsule Capsule|Suspended Animation Art Gallery|Infinite Jungle Canopy Haven|Hidden Subterranean Garden Sanctuary|Uncharted Underground Oasis|Dystopian Urban Underground Hive|Tropical Archipelago Sky Haven|Frozen Nebula Overlook|Mystical Eastern Palace Gardens|Sunken Galleon Graveyard|Urban Dreamscape Graffiti Sanctuary|Ancient Maya Astral Citadel|Victorian Chrono-Plaza|Eternal Roman Colosseum Nexus|Dalí-esque Surreal Dreamscape|Virtual Reality Cyber Mall}")

    response = requests.post(url=f'http://localhost:7860/sdapi/v1/txt2img', json=payload)
    
    images = response.json()['images']

    media = []
    for image_data in images:
        # Decodificar los datos de la imagen en bytes
        image_bytes = base64.b64decode(image_data)
        # Agregar la imagen a la lista de media
        media.append(telebot.types.InputMediaPhoto(media=image_bytes))
    bot.send_media_group(call.message.chat.id, media)

    estado = not estado
    
    keyboard = get_keyboard('principal')
    bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id, reply_markup=keyboard)


@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    global cantidad
    global adetailer
    global hiresfix
    global estado
    # print(call.message)

    if call.data == 'imagine_callback':
        
        reply_markup = call.message.json.get('reply_markup')
        generando = reply_markup["inline_keyboard"][0][0]["text"]
        if "Generar" in generando:
            bot.answer_callback_query(call.id, text="Intentando generar arte")
            estado = not estado
            keyboard = get_keyboard('principal')
            bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id, reply_markup=keyboard)
            generate_images(call, call.message)
        else:
            bot.answer_callback_query(call.id, text="Estamos generando tu arte!")
        # Aquí puedes realizar alguna acción relacionada con 'imagine_callback'
        # we call echo_message

        
    elif call.data == 'cantidad_callback':
        bot.answer_callback_query(call.id, text="Elige la cantidad de imágenes a generar")
        keyboard = telebot.types.InlineKeyboardMarkup()
        button_list = []
        for i in range(1, 5):  # Agrega botones del 1 al 4 en una tabla de 2x2
            button_list.append(telebot.types.InlineKeyboardButton(text=str(i), callback_data=f"numero_{i}_callback"))
            if len(button_list) == 2:  # Agrega una fila cada dos botones
                keyboard.add(*button_list)
                button_list.clear()
        if button_list:  # Si queda un botón, agrégalo
            keyboard.add(*button_list)
        bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id, reply_markup=keyboard)
    elif call.data.startswith('numero_'):
        cantidad = int(call.data.split('_')[1])
        bot.answer_callback_query(call.id, text=f"Has seleccionado {cantidad} imágenes")
        keyboard = get_keyboard('principal')
        bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id, reply_markup=keyboard)
    elif call.data == 'adetailer_callback':
        adetailer = not adetailer
        bot.answer_callback_query(call.id, text=f"Has seleccionado {str(adetailer)}")
        keyboard = get_keyboard('principal')
        bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id, reply_markup=keyboard)
    elif call.data == 'hrfix_callback':
        hiresfix = not hiresfix
        bot.answer_callback_query(call.id, text=f"Has seleccionado {str(hiresfix)}")
        keyboard = get_keyboard('principal')
        bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id, reply_markup=keyboard)
    else:
        bot.answer_callback_query(call.id, text="Devolución de llamada no reconocida")

    # Introduce una pequeña espera para evitar enviar demasiadas peticiones seguidas
    # loop = asyncio.get_event_loop()
    # asyncio.run_coroutine_threadsafe(sleep_and_respond(call), loop)
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    asyncio.run_coroutine_threadsafe(sleep_and_respond(call), loop)

async def sleep_and_respond(call):
    await asyncio.sleep(1)  # Wait for 1 second before responding
    await bot.answer_callback_query(call.id, text="¡Listo!")

@bot.message_handler(func=lambda message: True)
def echo_message(message):
    if message.text.startswith("/imagine "):
        message_text = message.text.replace("/imagine ", "").strip()
        keyboard = get_keyboard('principal')
        miMensaje = bot.reply_to(message, f'Qué te gustaría hacer con tu Prompt?\n*Prompt:*\n```{message_text}```', reply_markup=keyboard, parse_mode='markdown')
    else:
        print('nada')


print('Iniciando bot')
bot.infinity_polling()